<a href="https://colab.research.google.com/github/kyileiaye2021/HistoGPT/blob/main/histogpt_for_small_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup environment

In [ ]:
# install openslide dependencies
!sudo apt-get install openslide-tools
!sudo apt-get install python-openslide
!pip install openslide-python

In [ ]:
# install flamingo and histogpt
!pip install flamingo-pytorch --no-deps
!pip install git+https://github.com/marrlab/HistoGPT.git

In [ ]:
# check whether to use a gpu or cpu
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# Mounting the Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Download model into Google Drive
Change the drive file path to download the HistoGPT model and CTransPath image encoder.

In [ ]:
# download small/medium histogpt weights
# Already done if you've downloaded it and store it in google drive before
!wget "https://huggingface.co/marr-peng-lab/histogpt/resolve/main/histogpt-1b-6k-pruned.pth" -O /content/drive/MyDrive/Teacher_Student_Network/Histo GPT/Histogpt_weights/histogpt-1b-6k-pruned.pth

In [ ]:
# download ctranspath weights
# Already done if you've downloaded it and store it in google drive before
!wget https://huggingface.co/marr-peng-lab/histogpt/resolve/main/ctranspath.pth?download=true -O /content/drive/MyDrive/Teacher_Student_Network/Histo GPT/Ctranspath_weights/ctranspath.pth

# Extract features
Slide images are first made patches of the size 128. Then, features are extracted from each patch with CTransPath image encoder. Then, those features are concatenated as slide features and saved as part of .h5 files. For OCT images,since they are 1 channel images, we converted them to RGB 3 channel images to be able to use as input for HistoGPT.

In [ ]:
# patch and extract features (i don't think we need this. we already did uni encoding features)
import os
import shutil
from histogpt.helpers.patching import main, PatchingConfigs

import cv2
import numpy as np
import matplotlib.pyplot as plt

# loading grayscale images
grayscale_img = cv2.imread('/content/LHC-36-Slide07_Section01_yp0_A.jpg', cv2.IMREAD_GRAYSCALE)

# converting the grayscale to rgb
rgb_img = cv2.cvtColor(grayscale_img, cv2.COLOR_GRAY2RGB)
saved_rgb_img_path = '/content/LHC-36-Slide07_Section01_yp0_A_converted.jpg' # saved rgb file path
cv2.imwrite(saved_rgb_img_path, rgb_img)

try:
  os.mkdir('/content/slide_folder')
  os.mkdir('/content/save_folder')
  shutil.move(saved_rgb_img_path, "/content/slide_folder/LHC-36-Slide07_Section01_yp0_A_converted.jpg")
except Exception:
  pass

configs = PatchingConfigs()
configs.slide_path = '/content/slide_folder' # '/content/drive/MyDrive/Teacher_Student_Network/Split Dataset/he_val/H&E'
configs.save_path = '/content/save_folder' # '/content/drive/MyDrive/Teacher_Student_Network/Histo GPT/Real H&E test dataset features'
configs.file_extension = '.jpg'
configs.model_path = '/content/drive/MyDrive/Teacher_Student_Network/Histo GPT/Ctranspath_weights/ctranspath.pth' # should be the image encoder weight (ctranspath used right now)
configs.patch_size = 128
configs.white_thresh = [170, 185, 175]
configs.edge_threshold = 2
configs.resolution_in_mpp = 0.0
configs.downscaling_factor = 1.0
# configs.save_patch_images = True
configs.save_tile_preview = True
configs.batch_size = 16
# Use all slides in the folder
configs.split = [0, 1]  # Use all files

main(configs)

# Generate reports

In [ ]:
# configure histogpt and load weights
from transformers import BioGptConfig
from histogpt.models import HistoGPTForCausalLM, PerceiverResamplerConfig

histogpt = HistoGPTForCausalLM(BioGptConfig(), PerceiverResamplerConfig())
histogpt = histogpt.to(device)

PATH = '/content/drive/MyDrive/Teacher_Student_Network/Histo GPT/Histogpt_weights/histogpt-1b-6k-pruned.pth' # histogpt weight
state_dict = torch.load(PATH, map_location=device)
histogpt.load_state_dict(state_dict, strict=True)

# with h5py.File('/content/save_folder/h5_files/64px_ctranspath_1.0mpp_1.0xdown_normal/LG-62-Slide03_Section01_yp0_B.h5', 'r') as f:
#       features = f['feats'][:]
#       features = torch.tensor(features).unsqueeze(0).to(device)

In [ ]:
# generate text autoregressively
from histogpt.helpers.inference import generate

def generate_text(histogpt, prompt, image_feature, tokenizer):
  histogpt = histogpt.to(device)
  prompt = prompt.to(device)
  emb_tensor = features
  output = generate(
      model=histogpt,
      prompt=prompt,
      image=emb_tensor,
      length=256,
      top_k=40,
      top_p=0.95,
      temp=0.7,
      device=device
  )

  decoded = tokenizer.decode(output[0, 1:])
  return decoded

# For generating consistent output for histogpt
# def generate_text(histogpt, prompt, image_feature, tokenizer):
#   histogpt = histogpt.to(device)
#   prompt = prompt.to(device)
#   emb_tensor = features
#   output = generate(
#       model=histogpt,
#       prompt=prompt,
#       image=emb_tensor,
#       length=256,
#       top_k=1,
#       top_p=1.0,
#       temp=0.1,
#       device=device
#   )

#   decoded = tokenizer.decode(output[0, 1:])
#   return decoded

In [ ]:
# get text prompt and image features
import os
from PIL import Image
import textwrap
import matplotlib.pyplot as plt
import h5py
from transformers import BioGptTokenizer
res = []
res.append(('Image Path', 'HistoGPT Output (Prompt- Final Diagnosis:)'))
tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")

# text prompt
prompt = 'Final Diagnosis: '
prompt = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)

feature_folder = '/content/drive/MyDrive/Teacher_Student_Network/Histo GPT/Real H&E test dataset features/h5_files/128px_ctranspath_0.0mpp_1.0xdown_normal'
for h5_file in os.listdir(feature_folder):

  # image features
  if h5_file.endswith('.h5'):
    filename = os.path.join(feature_folder, h5_file)
    with h5py.File(filename, 'r') as f:
      features = f['feats'][:]
      features = torch.tensor(features).unsqueeze(0).to(device) # H&E image features

    # generate text
    decoded = generate_text(histogpt, prompt, features, tokenizer)
    image_path = h5_file.replace('.h5', '.png')
    image_path = os.path.join('/content/drive/MyDrive/Teacher_Student_Network/Split Dataset/he_val/H&E', image_path)
    image = Image.open(image_path)
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    print(textwrap.fill(decoded, width=64))

    # store the image paths and generated outputs
    res.append((h5_file, decoded))


The following code checks if the features for the image file is extracted and created as .h5 file. It is similar function to the above code. But, it iterates thru the image files and find it is in .h5 folder as well.

In [ ]:
# iterating thru image files and find it is in .h5 files as well
# get text prompt and image features
import os
from pathlib import Path
from PIL import Image
import textwrap
import matplotlib.pyplot as plt
import h5py
from transformers import BioGptTokenizer
res = []
res.append(('Image Path', 'HistoGPT Output (Prompt- Is there a basal cell carcinoma or basalioma?)'))
tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")

# text prompt
prompt = 'Is there a basal cell carcinoma?: '
prompt = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)

feature_folder = Path('/content/drive/MyDrive/Teacher_Student_Network/Histo GPT/Real H&E test dataset features/h5_files/128px_ctranspath_0.0mpp_1.0xdown_normal')
image_folder = Path('/content/drive/MyDrive/Teacher_Student_Network/Split Dataset/he_val/H&E')

h5_folder = {p.stem.lower(): p for p in feature_folder.glob('*.h5')} # hashmap to iterate more efficiently
# {.h5 filename without ext : .h5 full path}

for image_path in image_folder.iterdir():

  h5_file = image_path.stem.lower() # only image name with ext name stripped off

  # some of the features are not generated. so we need to check which images have .h5 features.
  if h5_file in h5_folder:
    h5_path = h5_folder[h5_file]
    with h5py.File(h5_path, 'r') as f:
      features = f['feats'][:]
      features = torch.tensor(features).unsqueeze(0).to(device) # H&E image features

    # generate text
    decoded = generate_text(histogpt, prompt, features, tokenizer)
    image_path = os.path.join(image_folder, image_path)
    image = Image.open(image_path)
    plt.imshow(image)
    plt.axis('off')
    plt.show()
    print(textwrap.fill(decoded, width=64))

    # store the image paths and generated outputs
    res.append((h5_file, decoded))

  # if there is no feature extracted for the image, we just add placeholder
  else:
    res.append((h5_file, "Feature not extracted."))




In [ ]:
# save the outputs in csv file
import csv

save_file = '/content/drive/MyDrive/Teacher_Student_Network/Histo GPT/HistoGPT outputs.csv'
with open(save_file, 'w', newline='') as f:
  writer = csv.writer(f)
  writer.writerows(res)

### For running histogpt for a single image

In [ ]:

# view image and print text
# from openslide import OpenSlide
from PIL import Image
import textwrap
import matplotlib.pyplot as plt
import h5py
from transformers import BioGptTokenizer

jpg_path = "/content/slide_folder/LHC-36-Slide07_Section01_yp0_A_converted.jpg"
image = Image.open(jpg_path)
plt.imshow(image)
plt.axis('off')
plt.show()

tokenizer = BioGptTokenizer.from_pretrained("microsoft/biogpt")
print()
# prompt = "Final diagnosis:"
prompt = 'Is there a basal cell carcinoma or basalioma?: '
prompt = torch.tensor(tokenizer.encode(prompt)).unsqueeze(0).to(device)
with h5py.File('/content/save_folder/h5_files/128px_ctranspath_0.0mpp_1.0xdown_normal/LHC-36-Slide07_Section01_yp0_A_converted.h5', 'r') as f:
      features = f['feats'][:]
      features = torch.tensor(features).unsqueeze(0).to(device)
      print(features.shape)

decoded = generate_text(histogpt, prompt, features, tokenizer)
print(textwrap.fill(decoded, width=64))

### Storing Good H&E Images in a folder

In [ ]:
# load csv file

# iterate thru the image folder
#   find if the curr image is in the csv file
#     check if it is good
#       store it in the new folder
import os
import csv
import collections

csv_map = collections.defaultdict(str)
separated_csv_file = "/content/drive/MyDrive/Teacher_Student_Network/Histo GPT/Test_Set_image_file_paths.csv"
all_image_path = "/content/drive/MyDrive/Teacher_Student_Network/Split Dataset/he_val/H&E"
good_image_path = "/content/drive/MyDrive/Teacher_Student_Network/Histo GPT/Good Real H&E images"
count = 0

#### CREATE A DICT FOR CSV FILE TO EXTRACT ROWS AND COLS FROM CSV FILE EFFECTIVELY #####
# read all rows in csv file and add the properties in a hashmap
# hashmap = {filename: [(BCC or No BCC), (Good or Bad)]}
with open(separated_csv_file, 'r') as f:
  reader = csv.DictReader(f)

  # for all other rows, add the filename and its properties in the dict
  for row in reader:
    filename = row['Image files']
    label = row['Good or Bad Examples (Modified)']
    csv_map[filename] = label
    count += 1

print(count)
print(csv_map)

##### MOVING GOOD IMGAES TO A NEW FOLDER TO GET CLEAN DATASET ######
good_images_num = 0
for filename in os.listdir(all_image_path):
  if filename in csv_map:
    if csv_map[filename] == 'Good':
      shutil.copy(os.path.join(all_image_path, filename), good_image_path)
      good_images_num += 1

print(good_images_num)

### Storing Good OCT images in a folder

In [ ]:
oct_image_path = "/content/drive/MyDrive/Teacher_Student_Network/Histo GPT/Good paired OCT images"
all_oct_image_path = "/content/drive/MyDrive/Teacher_Student_Network/Split Dataset/he_val/OCT"

good_oct_images_num = 0
for filename in os.listdir(all_oct_image_path):
  basename, ext = os.path.splitext(filename)
  basename = basename.replace("real_A", "real_B")

  oct_filename = basename + ext
  if oct_filename in csv_map:
    if csv_map[oct_filename] == 'Good':
      shutil.copy(os.path.join(all_oct_image_path, filename), oct_image_path)
      good_oct_images_num += 1

print(good_oct_images_num)